### Run image_processing module

In [1]:
import os
import glob
import logging
import concurrent.futures
from tqdm import tqdm

from sturdy_barnacle.db_utils import DatabaseManager
from sturdy_barnacle.image_processor import ImageProcessor

All table names are validated and safe.


In [2]:
num_workers = max(os.cpu_count() - 4, 1)

logging.basicConfig(
    filename="image_processing.log",
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

db = DatabaseManager()

def process_image(image_path: str) -> None:
    """
    Process an image if it hasn’t been processed. 
    Skips it if already processed and logs any errors.
    """
    try:
        if db.is_image_processed(image_path):
            logging.info(f"Skipping {image_path}, already processed.")
            return

        processor = ImageProcessor(image_path, db)
        processor.process_and_store()

    except Exception as e:
        logging.error(f"Error processing {image_path}: {e}")

In [3]:
# IMAGE_DIR = "/Users/mamduhzabidi/Pictures/Photos Library.photoslibrary/originals/0/"
# IMAGE_DIR = "/Users/mamduhzabidi/Pictures/Photos Library.photoslibrary/originals/1/"
IMAGE_DIR = "/Users/mamduhzabidi/Pictures/Photos Library.photoslibrary/originals/2/"
image_paths = glob.glob(f"{IMAGE_DIR}/*.jpeg")

In [4]:
with concurrent.futures.ThreadPoolExecutor(max_workers=num_workers) as executor:
    with tqdm(total=len(image_paths), desc="Processing Images") as pbar:
        futures = {executor.submit(process_image, img): img for img in image_paths}
        for future in concurrent.futures.as_completed(futures):
            pbar.update(1)

Processing Images: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1032/1032 [00:00<00:00, 1730.94it/s]
